### Revision: 

- Includes seasional attributes for environment.''' 

- It uses demand-forecasting-kernels-only training data match the demand using week of the year and day of the week.   

- Data: Store 1, Item 2, Year 2017

#### Prepare dataset to simulate the local store environment

In [8]:
from gym import spaces
import gym
import pandas as pd 
import numpy as np 
import time
import warnings
warnings.filterwarnings("ignore")

m = 60
class LocalStoreEnv(gym.Env):
    """
    Description:
        State  : day of the week
        Reward : 1 if it meets the demand -1 if exceeds the demand -1 if it doesn't meet the demand
        Inventory: capacity of the shelf default = 10
    """

    metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": m}

     # returns demand value for a given day of week and week of year from existing historical load

    def demand(self, day_of_week,week_of_year):
        demand_value_from_table =self.demand_data[self.demand_data.week == week_of_year][self.demand_data.day == day_of_week]['sales'].iat[0] 
        #return np.random.randint(demand_value_from_table-3,demand_value_from_table+4)
        return np.random.poisson(demand_value_from_table)
        #return demand_value_from_table
    
    def __init__(self):
        self.tries = 0
        
        ############################
        #Load Demand Seasional Data#
        ############################
        
        self.demand_data = pd.read_csv('store1_item1_2017_sales.csv')
        self.demand_data['date'] = self.demand_data['date'].astype('datetime64[ns]')
        self.demand_data['week'] = self.demand_data.date.dt.week
        self.demand_data['day'] = self.demand_data.date.dt.weekday
        self.demand_data['sales'] = self.demand_data.sales.astype(int)
        
        ############################
        #End of Loading Demand Data#
        ############################
        
        self.start_time =  time.time()
        self.log_time = time.time()
        self.duration = 0
        self.inventory = np.random.choice(np.arange(0,m+1))
        self.action_space = spaces.Discrete(m+1)
        self.observation_space = spaces.Discrete((m+1)*7*52)
        self.day = np.random.choice((0,1,2,3,4,5,6))
        self.week = np.random.choice((52,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51))
        #self.day = np.random.choice((0,1,2))
        self.state = (self.inventory,self.day,self.week)
        self.reset()
        print("init")

        
    def step(self, action):
        print_interval = 1*30
        end_week_of_the_year = 52
        end_day_of_the_week = 6
        end_of_the_year = (52,6)
        #print_log = False
        print_log = (time.time() - self.log_time) >= print_interval
        if(print_log):
         self.log_time = time.time()
        #print_log = True
        self.tries = self.tries+1
        day_of_yesterday = self.state[1]-1
        week_of_yesterday = self.state[2]
        if(day_of_yesterday == -1):
            day_of_yesterday = 6
            week_of_yesterday = week_of_yesterday - 1
        print_log = False
        if(print_log):
            print("Tries : {}, Yesterday Inventory :{} ,Today's Demand: {} , Today's order : {}".format(self.tries,self.inventory,self.demand(self.day,self.week),action))
        yesterday_inv = 0
        if(self.state[0] >0):
            yesterday_inv = self.state[0]
        inv_today  = yesterday_inv+action # -self.demand(yesterday)
        reward = 0
        demand = self.demand(self.state[1],self.state[2])
        if(abs(inv_today-demand) > 5):
            reward = -1

        reward = self.get_rewards(inv_today-demand,inv_today,demand)
        tomorrow_inv = inv_today-self.demand(self.day,self.week)
        self.inventory=max(min(tomorrow_inv,m),0)
        
        if(self.state[1]<end_day_of_the_week):
            self.day=self.state[1]+1
        else:
            self.day=0
            if(self.state[2]<end_week_of_the_year):
                self.week = self.week+1
            else:
                self.week = 1

        self.state = (self.inventory,self.day,self.week)
        if(print_log):
            print(" Tomorrow's Inventory :{} , Today's inventory : {} ,Today's Demand: {}, reward {}".format(self.inventory, inv_today,demand,reward))
        return self.state, reward, False, {}
    def get_rewards(self,diff,inventory,demand):
        if(diff>0):
            return -diff
        else:
            return diff
    def reset(self):
        return self.state

    def render(self, mode="human"):
        print("Inventory :{} , Demand: {}".format(self.inventory,self.demand(self.day,self.week)))

    def close(self):
        print("close")

In [19]:
env = LocalStoreEnv()
env.state

init


(31, 3, 28)

In [20]:
env.step(0)

((5, 4, 28), -10, False, {})